#### 1. Loading Processed Data

In [1]:
import numpy as np
import os

# Path relative to notebook directory
base_path = "../data/processed"

X_train = np.load(os.path.join(base_path, "X_train_processed.npy"))
X_test = np.load(os.path.join(base_path, "X_test_processed.npy"))
y_train = np.load(os.path.join(base_path, "y_train_encoded.npy"))
y_test = np.load(os.path.join(base_path, "y_test_encoded.npy"))

print(X_train.shape, y_train.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed\\X_train_processed.npy'

#### 2. Initial Setup

##### 2.1 Split training/validation sets

In [ ]:
from sklearn.model_selection import train_test_split
# split the data into 10% validation set and 90% training set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
)


In [ ]:
from tensorflow.keras.utils import to_categorical # Pylance might still warn, but it's correct
# One hot encoding to handle categorical formatting
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

##### 2.2 Building CNN model

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


e:\Ashwin\importantFiles\Programming Projects\WASL-translator\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 686,365 (2.62 MB)

 Trainable params: 686,365 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

#### 3. Training Model

##### 3.1 Early Stopping, checkpoints, Optimisation

In [2]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,      # stop if no improvement for 5 epochs
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    filepath=f"best_model_{run_name}.keras",
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)
learning_rate = 0.001  # Try 0.001 or 0.0005
optimizer = Adam(learning_rate=learning_rate)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

NameError: name 'run_name' is not defined

##### 3.2 Fitting model

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=(X_val, y_val),
    epochs=15,
    batch_size=64,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

### 4. Evaluation
###### inclues: Accuracy, DR/FAR, Precision/Recall/F1, ROC Area, Confusion Matrix

In [ ]:
# model.evaluate(X_test, y_test)

##### 4.1 Functions

In [ ]:
import csv, os, datetime

def save_run_metrics(run_name, accuracy, precision, recall, f1, FAR, roc_area, auc, csv_path="model_results.csv"):
    file_exists = os.path.isfile(csv_path)

    with open(csv_path, mode="a", newline="") as f:
        writer = csv.writer(f)

        # Header on first creation
        if not file_exists:
            writer.writerow([
                "timestamp", "run_name",
                "accuracy", "precision", "recall",
                "f1", "false_acceptance_rate",
                "roc_area", "auc"
            ])

        writer.writerow([
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            run_name,
            accuracy,
            precision,
            recall,
            f1,
            FAR,
            roc_area,
            auc
        ])


##### 4.2 Evaluating and Saving

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

print("\n Running evaluation...")

# Predict probabilities & labels
y_prob = model.predict(X_test_proc)
y_pred = np.argmax(y_prob, axis=1)

# Convert labels to one-hot for ROC
y_test_onehot = tf.keras.utils.to_categorical(y_test_enc, num_classes=y_prob.shape[1])

# --- Core Metrics ---
accuracy = accuracy_score(y_test_enc, y_pred)
precision = precision_score(y_test_enc, y_pred, average="macro")
recall = recall_score(y_test_enc, y_pred, average="macro")   # Detection Rate
f1 = f1_score(y_test_enc, y_pred, average="macro")

# ROC/AUC (multi-class)
try:
    auc = roc_auc_score(y_test_onehot, y_prob, multi_class="ovr")
except ValueError:
    auc = np.nan

roc_area = auc  # same metric, requested both names

# Confusion Matrix
cm = confusion_matrix(y_test_enc, y_pred)

# False Acceptance Rate (FAR)
FP = cm.sum(axis=0) - np.diag(cm)
TN = cm.sum() - (FP + (cm.sum(axis=1) - np.diag(cm)) + np.diag(cm))
FAR = np.mean(FP / (FP + TN + 1e-6))


# --- Print Results ---
print("\n Model Evaluation Metrics")
print(f"Accuracy:              {accuracy:.4f}")
print(f"Precision:             {precision:.4f}")
print(f"Recall (Detection Rate): {recall:.4f}")
print(f"F1 Score:              {f1:.4f}")
print(f"False Acceptance Rate: {FAR:.4f}")
print(f"ROC Area:              {roc_area:.4f}")
print(f"AUC:                   {auc:.4f}")

print("\n Classification Report")
print(classification_report(y_test_enc, y_pred))
# Save metrics to CSV
save_run_metrics(
    run_name,
    accuracy,
    precision,
    recall,
    f1,
    FAR,
    roc_area,
    auc
)

print(f"\n Metrics saved for run: {run_name}")



##### 4.2 Print and Save

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(cm, annot=False, cmap="viridis")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()
plt.savefig("confusionMatrix.png", dpi=300)

In [ ]:
model.save("../models/asl_cnn_model.h5")